In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from glob import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import h5py

from PIL import Image

In [3]:
data_dir = 'data/original/'
out_dir = 'data/'
image_size = (224, 224)

In [4]:
filepaths = [f for f in glob(os.path.join(data_dir, '**'), recursive=True)
             if os.path.isfile(f) and os.path.splitext(f)[-1].lower() in ['.jpg', '.png']]
print(filepaths[:5])

['data/original/No_RA/IMG_3445.JPG', 'data/original/No_RA/IMG_2914.JPG', 'data/original/No_RA/IMG_2855.JPG', 'data/original/No_RA/IMG_1388.JPG', 'data/original/No_RA/IMG_0903.JPG']


In [6]:
_old_s = ''
def print_progress(s):
    global _old_s
    print(' ' * len(_old_s), end='\r')
    print(s, end='\r', flush=True)
    _old_s = s

def load_images(paths):
    N = len(paths)
    X = np.zeros((N, image_size[0], image_size[1], 3))

    for i, path in enumerate(paths):        
        print_progress(f'[{i+1}/{N}] {path}')
        img = Image.open(path)
        img = img.resize(image_size)
        X[i] = np.array(img)
    
    return X

X = load_images(filepaths)

In [7]:
relpaths = [os.path.relpath(f, data_dir) for f in filepaths]
classnames = [f.split('/', 1)[0] for f in relpaths]
filenames = [f.split('/', 1)[1] for f in relpaths]

In [16]:
h5f = h5py.File(os.path.join(out_dir, f'RockAI_images_{image_size[0]}x{image_size[1]}.h5'), 'w')
h5f['X'] = X
h5f['filename'] = np.array(filenames).astype('S10')
h5f['classname'] = np.array(classnames).astype('S10')
h5f.close()